In [ ]:
from itertools import product
import pickle

import ipywidgets as ipw
import numpy as np

from matplotlib import pyplot as plt
from astropy.table import Table

from stellarphot.analysis.transit_fitting import TransitModelFit
from stellarphot.visualization.transit_plots import *
from stellarphot.visualization.fits_opener import FitsOpener

from astropy.timeseries import BinnedTimeSeries, TimeSeries, aggregate_downsample
from astropy.time import Time
from astropy.table import Table 
from astropy import units as u

### 0. Get some data

In [ ]:
fo = FitsOpener(title="Select your photometry/flux file", filter_pattern=["*.csv", "*.fits"])
fo2 = FitsOpener(title="Select your TESS info file", filter_pattern=["*.pickle"])
passband = ipw.Dropdown(description="Filter", options=['gp', 'ip'], disabled=True)
box = ipw.VBox()

def update_filter_list(change):
    tab = Table.read(fo.path)['filter']
    passband.options = sorted(set(tab))
    passband.disabled = False
    passband.value = passband.options[0]
    
fo.register_callback(update_filter_list)
box.children = [fo.file_chooser, fo2.file_chooser, passband]
box

In [ ]:
# 👉 File with photometry, including flux
photometry_file = fo.path

# 👉 File with exoplanet info in
tess_info_output_file = fo2.path

phot_filter = passband.value

In [ ]:
phot_filter

In [ ]:
photometry = Table.read(photometry_file)

with open(tess_info_output_file, 'rb') as f:
    tess_info = pickle.load(f)

### You may need to alter some of the settings here

In [ ]:
# These affect the fitting that is done

#bin size in minutes
bin_size = 5 * u.min

# Keep the time of transit fixed?
keep_fixed_transit_time = True
transit_time_range = 60 * u.min

# Keep radius of planet fixed?

keep_fixed_radius_planet = False

# Keep radius of orbit fixed?

keep_fixed_radius_orbit = False

# Remove effects of airmas?
fit_airmass = False

# Remove effects of sky background?
fit_spp = False

# Remove effects of change in focus?
fit_width = False


In [ ]:
# Enter your object's period here
period = tess_info.period

# Enter the epoch here
epoch = tess_info.epoch  # Time(2458761.602894, scale='tdb', format='jd')

# Enter the duration below
duration = tess_info.duration

# Enter the transit depth here -- get the "ppm" value from ExoFOP-TESS
depth = tess_info.depth

# Enter object name
obj = 'TIC ' + tess_info.tic_id

In [ ]:
# These affect spacing of lines on final plot
high = 1.06
low = 0.82
scale = 0.15 * (high - low)
shift = -.72 * (high - low)

In [ ]:
target_star = photometry['id'] == 1

# No changes to the line below, it is grabbing the first time in the data series
then = Time(photometry['BJD'][target_star][0], scale='tdb', format='jd')

date_obs = photometry['date-obs'][target_star][0]
exposure_time = photometry['exposure'][target_star][0]

Looks like we need to normalize the data first.....

In [ ]:
band_filter = photometry['filter'] == phot_filter

target_and_filter = target_star & band_filter

In [ ]:
photometry = photometry[target_and_filter]

In [ ]:
photometry

In [ ]:
cycle_number = int((then - epoch) / period + 1)
that_transit = cycle_number * period + epoch
that_transit

In [ ]:
start = that_transit - duration / 2
mid = that_transit
end = that_transit + duration / 2

after_transit = (photometry['BJD'] - 2400000) > end.jd

outside_transit = (photometry['BJD'] < start.jd) | (photometry['BJD'] > end.jd)

normalization_factor = np.nanmean(1 / photometry['relative_flux'][outside_transit])
normalized_flux = photometry['relative_flux'] * normalization_factor

### Bin Data

Need
* data table
* start
* end
* bin_size

In [ ]:
t_ob = Time(photometry['BJD'], scale='tdb', format='jd')
ts = TimeSeries([normalized_flux, photometry['airmass'], photometry['xcenter'], photometry['sky_per_pix_avg'], photometry['width']], time=t_ob)
ts2 = TimeSeries([normalization_factor * photometry['relative_flux_error']], time=t_ob)

first_time =  photometry['BJD'][0] - 2400000
last_time = photometry['BJD'][-1] - 2400000

def add_quad(x):
    try:
        n = len(x)
    except TypeError:
        n = 1
    return np.sqrt(np.nansum(x**2)) / n

binned = aggregate_downsample(ts, time_bin_size=bin_size)
binned2 = aggregate_downsample(ts2, time_bin_size=bin_size, aggregate_func=add_quad)

#binned_time = BinnedTimeSeries(photometry['BJD'], time_bin_start=first_time, time_bin_end=last_time, time_bin_size=bin_size)

### 1. Create the model object

In [ ]:
mod = TransitModelFit()

### 2. Load some data

Here we will just load times and normalized flux. You can also set width, spp (sky per pixel) and airmass. The only two that must be set are times and flux.

If you have set `mod.spp`, `mod.width` or `mod.airmass` then those things will be included in the fit. Otherwise, they are ignored.

THE WEIGHTS ARE IMPORTANT TO INCLUDE

In [ ]:
not_empty = ~np.isnan(binned['relative_flux'])

mod.times = (np.array(binned['time_bin_start'].value) - 2400000)[not_empty]
mod.data = binned['relative_flux'].value[not_empty]
mod.weights = 1 / (binned2['relative_flux_error'].value)[not_empty]



### 3. Set up the model

You should be able to get the parameters for this from TTF. There are more parameters you can set; `shift-Tab` in the arguments to pull up the docstring, which lists and explains them all.

In [ ]:
mod.setup_model(t0=mid.jd - 2400000,         # midpoint, BJD
                depth=depth,       # parts per thousand
                duration=duration.to('day').value,  # days
                period=period.to('day').value,    # days
               )

### 3.25 Set up airmass, etc

In [ ]:
mod.airmass = np.array(binned['airmass'])[not_empty]
mod.width = np.array(binned['width'])[not_empty]
mod.spp = np.array(binned['sky_per_pix_avg'])[not_empty]

### 3.5 Constrain the fits if you want

#### Exoplanet parameters

In [ ]:
mod.model.t0.bounds = [mid.jd - 2400000 - transit_time_range.to('day').value/2, mid.jd - 2400000 + transit_time_range.to('day').value/2]
mod.model.t0.fixed = keep_fixed_transit_time
mod.model.a.fixed = keep_fixed_radius_orbit
mod.model.rp.fixed = keep_fixed_radius_planet



#### Detrending parameters

In [ ]:
mod.model.spp_trend.fixed = not fit_spp
mod.model.airmass_trend.fixed = not fit_airmass
mod.model.width_trend.fixed = not fit_width

In [ ]:
detrended_by = []
if fit_airmass:
    detrended_by.append('Airmass')

if fit_spp:
    detrended_by.append('SPP')

if fit_width:
    detrended_by.append('Wdith')

detrended_by = ("Detrended by: " + ",".join(detrended_by)) if detrended_by else "No detrending"
        

### 4. Run the fit

In [ ]:
mod.fit()

### 5. Let's try a plot....

In [ ]:
plt.plot(mod.times, mod.data, '.')
plt.plot(mod.times, mod.model_light_curve())
plt.vlines(start.jd - 2400000, 0.98, 1.02, colors='r', linestyle='--', alpha=0.5)
plt.vlines(end.jd - 2400000, 0.98, 1.02, colors='r', linestyle='--', alpha=0.5)
plt.title('Data and fit')
plt.grid()

In [ ]:
mod.model

In [ ]:
# mod._fitter.fit_info

In [ ]:
flux_full_detrend = mod.data_light_curve(detrend_by='all') 
flux_full_detrend_model = mod.model_light_curve(detrend_by='all') 

In [ ]:
rel_detrended_flux = flux_full_detrend / np.mean(flux_full_detrend)

rel_detrended_flux_rms = np.std(rel_detrended_flux)
rel_model_rms = np.std(flux_full_detrend_model - rel_detrended_flux)

rel_flux_rms = np.std(mod.data)

In [ ]:
airmass = photometry['airmass'] / np.mean(photometry['airmass'])
x = photometry['xcenter'] / np.mean(photometry['xcenter'])
y = photometry['ycenter'] / np.mean(photometry['ycenter'])
comp_counts = photometry['comparison counts'] / np.mean(photometry['comparison counts'])
sky_per_pix = photometry['sky_per_pix_avg'] / np.mean(photometry['sky_per_pix_avg'])
width = photometry['width'] / np.mean(photometry['width'])

In [ ]:
scale_airmass = scale_and_shift(airmass, scale, 0.75 * shift, pos=False)
scale_x = scale_and_shift(x, scale, shift, pos=True)
scale_y = scale_and_shift(y, scale, shift, pos=True)
scale_sky_pix = scale_and_shift(sky_per_pix, scale, shift, pos=True)
scale_counts = scale_and_shift(comp_counts, scale, shift, pos=True)
scale_width = scale_and_shift(width, scale, shift, pos=True)

In [ ]:
grid_y_ticks = np.arange(low, high, 0.02)

In [ ]:
#(RMS={rel_flux_rms:.5f})

plt.figure(figsize=(8,11))
fig, ax = plt.subplots(1, 1, figsize=(8, 11))

plt.plot(photometry['BJD'] - 2400000, normalized_flux, 'b.', label=f"rel_flux_T1 (RMS={rel_flux_rms:.5f})", ms=4)

plt.plot(mod.times, flux_full_detrend - 0.04, '.', c='r', ms=4,
            label=f"rel_flux_T1 ({detrended_by})(RMS={rel_detrended_flux_rms:.5f}), (bin size={bin_size} min)")

plt.plot(mod.times, flux_full_detrend - 0.08, '.', c='g', ms=4,
            label=f"rel_flux_T1 ({detrended_by} with transit fit)(RMS={rel_model_rms:.5f}), (bin size={bin_size})")
plt.plot(mod.times, flux_full_detrend_model - 0.08, c='g', ms=4,
            label=f"rel_flux_T1 Transit Model ([P={mod.model.period.value:.4f}], "
                  f"(Rp/R*)^2={(mod.model.rp.value)**2:.4f}, \na/R*={mod.model.a.value:.4f}, "
                  f"[Tc={mod.model.t0.value + 2400000:.4f}], "
                  f"[u1={mod.model.limb_u1.value:.1f}, u2={mod.model.limb_u2.value:.1f})")

plt.plot(photometry['BJD'] - 2400000, scale_counts, '.', c='brown', label='tot_C_cnts (arbitrarily scaled and shifted)', alpha=0.5, ms=4)
plt.plot(photometry['BJD'] - 2400000, scale_airmass, 'c-', label="AIRMASS (arbitrarily scaled and shifted)", ms=4)
plt.plot(photometry['BJD'] - 2400000, scale_sky_pix, c='gold', label='Sky/Pixel_T1 (arbitrarily scaled and shifted)', ms=4)
plt.plot(photometry['BJD'] - 2400000, scale_width, '-', c='gray', label="Width_T1 (arbitrarily scaled and shifted)", ms=4)
plt.plot(photometry['BJD'] - 2400000, scale_x, '-', c='pink', label="X(FITS)_T1 (arbitrarily scaled and shifted)", ms=4)
plt.plot(photometry['BJD'] - 2400000, scale_y, '-', c='lightblue', label="Y(FITS)_T1 (arbitrarily scaled and shifted)", ms=4)


plt.vlines(start.jd - 2400000, low, 1.025, colors='r', linestyle='--', alpha=0.5)
plt.vlines(end.jd - 2400000, low, 1.025, colors='r', linestyle='--', alpha=0.5)
plt.text(start.jd - 2400000, low+0.0005, f'Predicted\nIngress\n{start.jd-2400000-int(start.jd - 2400000):.3f}', horizontalalignment='center',c='r')
plt.text(end.jd - 2400000, low+0.0005, f'Predicted\nEgress\n{end.jd-2400000-int(end.jd - 2400000):.3f}', horizontalalignment='center',c='r')

# plt.vlines(start + 0.005, low, 1, colors='darkgray', linestyle='--', alpha=0.5)
# plt.text(start + 0.005, low+0.001, f'Left\n{start-int(start)+0.005:.3f}', horizontalalignment='center',c='darkgray')
# plt.vlines(end - 0.005, low, 1, colors='darkgray', linestyle='--', alpha=0.5)
# plt.text(end - 0.005, low+0.001, f'Rght\n{end-int(end)-0.005:.3f}', horizontalalignment='center',c='darkgray')


plt.ylim(low, high)
plt.xlabel('Barycentric Julian Date (TDB)', fontname='Arial')
plt.ylabel('Relative Flux (normalized)', fontname='Arial')
plt.title(f'{obj}.01   UT{date_obs}\nPaul P. Feder Observatory 0.4m ({phot_filter} filter, {exposure_time} exp, fap 10-25-40)', 
          fontsize=14, fontname='Arial')
plt.legend(loc="upper center", frameon=False, fontsize=8, bbox_to_anchor=(0.6, 1.0))
ax.set_yticks(grid_y_ticks)
plt.grid()

plt.savefig(f'TIC{tess_info.tic_id}-01_20200701_Paul-P-Feder-0.4m_gp_lightcurve.png', facecolor='w')

In [ ]:
mod.n_fit_parameters

In [ ]:
mod._all_detrend_params

In [ ]:
def evaluate_fits(mod):
    BICs = []
    settings = []
    all_trendable = mod._all_detrend_params
    tf_sequence = product([True, False], repeat=len(all_trendable))
    for fixed in tf_sequence:
        this_summary = []
        for param, fix in zip(all_trendable, fixed):
            trend_mod = getattr(mod.model, f"{param}_trend")
            if fix:
                setattr(mod.model, f"{param}_trend", 0.0)
            trend_mod.fixed = fix
            this_summary.append(f"{param}: {not fix}")

        settings.append(', '.join(this_summary))
        mod.fit()
        BICs.append(mod.BIC)
    return Table(data=[settings, BICs], names=['Fit this param?', 'BIC'])

In [ ]:
bic_table = evaluate_fits(mod)
bic_table.sort('BIC')
bic_table